# HW 2: Corporate Bond Pricing (due by 9.21 Fri)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2018`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2018/HW2/HW2.ipynb`__  (Please use the same name for repository and ipynb file)
* Adding more code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [20]:
import numpy as np
import time

In [21]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [22]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(9.930430700363729, 10.0)

In [23]:
# Put your code here to price the corporate bond

def corp_bond(mat=10, def_rate=0.1, rf_rate=0.03, recovery=0.3, n_sample=10000):
    ### generation of a default time sequence
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U/def_rate)
    ### if default_time < mat, then the end_value is recovery;otherwise, 1
    for i in default_time:
      if i <= mat:
        price = np.exp(-rf_rate*default_time)*recovery
      else:
        price = np.exp(-rf_rate*mat) 

    return price.mean()

# Call your function
corp_bond()

# Find the mean and std by calling the function 100 times. 
simu_time = 100
bond_price_set = []
start = time.clock()

for i in range(simu_time):
    bond_price_set.append(corp_bond())

end = time.clock()

print(end-start)

print('mean:',np.mean(bond_price_set),'standard deviation:',np.std(bond_price_set))


93.50322709471038
mean: 0.4798988536902587 standard deviation: 0.07159165821066814


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [27]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(10.0, 10.0)

In [34]:
# No include the two new features: `antithetic` and `match`

# No include the two new features: `antithetic` and `match`

def corp_bond_cv(mat=10, def_rate=0.1, rf_rate=0.03, recovery=0.3, n_sample=10000, antithetic=True, match=True):
    ### <--
    if(antithetic):
        m_sample = np.int(n_sample/2)
        U = np.random.uniform(size=m_sample)
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
        exp_value = np.mean(np.where(default_time<mat, recovery*np.exp(-rf_rate*(default_time)), np.exp(-rf_rate*mat)))
        
    if(match):
        U = np.random.uniform(size=n_sample)
        default_time = -(1/def_rate)*np.log(U)
        default_time += 1/def_rate-default_time.mean()
        exp_value = np.mean(np.where(default_time<mat, recovery*np.exp(-rf_rate*(default_time)), np.exp(-rf_rate*mat)))
        
    if(antithetic and match):
        m_sample = np.int(n_sample/2)
        U = np.random.uniform(size=m_sample)
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
        default_time += 1/def_rate-default_time.mean()
        exp_value = np.mean(np.where(default_time<mat, recovery*np.exp(-rf_rate*(default_time)), np.exp(-rf_rate*mat)))
    return exp_value
# Find the mean and std by calling the function 100 times for (i) antithetic (ii) match and (iii) both
simu_time = 100
antithetic_price_set = []
match_price_set = []
both_price_set = []
for i in range(simu_time):
    antithetic_price_set.append(corp_bond_cv(match=False))
    match_price_set.append(corp_bond_cv(antithetic=False))
    both_price_set.append(corp_bond_cv())

print('antithetic')
print('mean:',np.mean(antithetic_price_set),'standard deviation:',np.std(antithetic_price_set))
print('match')
print('mean:',np.mean(match_price_set),'standard deviation:',np.std(match_price_set))
print('both')
print('mean:',np.mean(both_price_set),'standard deviation:',np.std(both_price_set))



antithetic
mean: 0.4404834571238473 standard deviation: 0.0015033169338638172
match
mean: 0.4402744014055209 standard deviation: 0.0015138255262073198
both
mean: 0.44042006923018034 standard deviation: 0.0015365601888465467


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [36]:
### Put the analytic expression for the corporate bond price
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10
corp_bond_price = 3*(1-np.exp(-1.3))/13 + np.exp(-1.3)
print(corp_bond_price)

0.44040907156462505
